### Basic model:
Features: Income, Household composition, Property type
Predict installed pV per household (aggregation level: buurt, no time dependency) 

Income: CBS data '84799NED' (Kerncijfers wijken en buurten 2020)

In [ ]:
# pip install cbsodata

In [ ]:
import cbsodata
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#Read in (Kerncijfers wijken en buurten 2020
kerncijfers_2020 = '84799NED'
df_kerncijfers = pd.DataFrame(cbsodata.get_data(kerncijfers_2020))
df_kerncijfers.head()

Remove unusable items

In [ ]:
df_kerncijfers = df_kerncijfers[df_kerncijfers['Codering_3'].isna() == False]

Keep only the data on Buurt level

In [ ]:
#remove whitespaces from beginning and end of string column labels
df_kerncijfers = df_kerncijfers.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

is_buurt = df_kerncijfers['SoortRegio_2']=='Buurt'
df_kerncijfers = df_kerncijfers[is_buurt]

### Feature #1 - Income

Take a look at the number of buurten where the incomes are unknown

In [ ]:
fraction_unfilled_incomes = df_kerncijfers['GemiddeldInkomenPerInwoner_72'].isna().sum()/ df_kerncijfers['GemiddeldInkomenPerInwoner_72'].sum() *100
print("Income not specified in: %.0f" % fraction_unfilled_incomes, "% of the buurten. Removing these entries.")

In [ ]:
cutoff_income = 60
df_kerncijfers = df_kerncijfers[(df_kerncijfers['GemiddeldInkomenPerInwoner_72'].isna() == False)]

df_income_specified = df_kerncijfers[df_kerncijfers['GemiddeldInkomenPerInwoner_72'] < cutoff_income]
df_income_specified['GemiddeldInkomenPerInwoner_72'].hist(bins=54, figsize=(8, 6))

### Feature #2 - Household composition

In [ ]:
fraction_unfilled_huishoudensgroottes = df_kerncijfers['GemiddeldeHuishoudensgrootte_32'].isna().sum()/ df_kerncijfers['GemiddeldeHuishoudensgrootte_32'].sum() *100

print("Average size of household not specified in: %.0f" % fraction_unfilled_huishoudensgroottes, "% of the buurten. Removing these entries.")

In [ ]:
cut_off_household_size = 5
df_kerncijfers = df_kerncijfers[df_kerncijfers['GemiddeldeHuishoudensgrootte_32'].isna() == False]

df_average_household_size_specified = df_kerncijfers[df_kerncijfers['GemiddeldeHuishoudensgrootte_32']<cut_off_household_size]
df_average_household_size_specified['GemiddeldeHuishoudensgrootte_32'].hist(bins=140, figsize=(8, 6))

### Feature #3 - Percentage owned property (koopwoningen)

In [ ]:
fraction_unfilled_owned_property_percentage = df_kerncijfers['Koopwoningen_40'].isna().sum()/ df_kerncijfers['Koopwoningen_40'].sum() *100

print("Average percentage of owned properties not specified in: %.0f" % fraction_unfilled_owned_property_percentage, "% of the buurten.")


Remove the 'BU' from the buurtcode and rename the column name so we can combine the datasets later

In [ ]:
has_buurtcode_starting_with_BU = df_kerncijfers['Codering_3'].str.find('BU') == 0
buurtcodes_without_leading_BU = df_kerncijfers[has_buurtcode_starting_with_BU == False]
print("Number of buurten that don't start with 'BU': ", len(buurtcodes_without_leading_BU))

df_kerncijfers['CBS Buurtcode'] = pd.to_numeric(df_kerncijfers['Codering_3'].apply(lambda s:s.replace("BU","")))

In [ ]:
print("Duplicate buurtcodes: %.0f" % df_kerncijfers.duplicated(['CBS Buurtcode']).sum())
df_kerncijfers.astype({"CBS Buurtcode" : int})
df_kerncijfers.head()

Take a look at the # of households per buurt. We need this to be able to 'normalize' the installed pV

In [ ]:
print("Huishoudens totaal has: ", (df_kerncijfers['HuishoudensTotaal_28'].isna() == True).sum(), " empty items.")

In [ ]:
df_kerncijfers.info()

## Target variable - opgesteld vermogen

Load the data from the Enexis supplied data file.

In [ ]:
decentral_generation_072020 = '../Data/Enexis_decentrale_opwek_kv_(zon_pv)_01072020.csv'
df_decentral_generation = pd.read_csv(decentral_generation_072020,
                         sep                = ';',
                         decimal            = ',',
                         thousands          = '.',
                         encoding           = 'unicode_escape')        

Remove empty items

In [ ]:
df_decentral_generation = df_decentral_generation[df_decentral_generation['Opgesteld vermogen'].isna() == False]
df_decentral_generation = df_decentral_generation[df_decentral_generation['CBS Buurtcode'].isna() == False]

## Note: the unit of 'opgesteld vermogen' is kW

In [ ]:
cut_off_generation = 3000

df_decentral_generation_specified = df_decentral_generation[df_decentral_generation['Opgesteld vermogen'] < cut_off_generation]
df_decentral_generation_specified['Opgesteld vermogen'].hist(bins=53, figsize=(8, 6))

In [ ]:
print("Duplicate buurtcodes: %.0f" % df_decentral_generation.duplicated(['CBS Buurtcode']).sum())
df_decentral_generation.astype({"CBS Buurtcode" : int})

Check if the deduplication was successfull

### Combine the demographic data with the generation data

In [ ]:
print("Number of rows in 'kerncijfers': %.0f" % len(df_kerncijfers))
print("Number of rows in 'generation data': %.0f" % len(df_decentral_generation))

df = pd.merge(df_kerncijfers, df_decentral_generation, on="CBS Buurtcode", validate='one_to_one')
print("Number of rows in combined data set: %.0f" % len(df))


#### To do: check why not more rows match on buurt code

### Introduce a normalized column. normalized_opgesteld_vermogen = opgesteld_vermogen / #households

In [ ]:
df["normalized_opgesteld_vermogen"] = (df["Opgesteld vermogen"] / df["HuishoudensTotaal_28"])

In [ ]:
df.info()

In [ ]:
df.head()

## EDA - impact of demographic features on installed pv capacity

Creating the list of features

In [ ]:
metadata = pd.read_csv('../Data/84799NED_Metadata.csv',
                         sep                = ';',
                         decimal            = ',',
                         thousands          = '.',
                         encoding           = 'utf-8')  

In [ ]:
metadata.head()

In [ ]:
# features_ext = metadata[metadata['Variable type'] == 'Extensive']['Key'].to_list()

In [ ]:
# let's keep the number of households as is, to take account of the correlation of the size of the buurt with installed PV
# features_ext.remove('HuishoudensTotaal_28')

In [ ]:
# features_ext

In [ ]:
# Extensive features converted into intensive by deviding by number of households
# for feat in features_ext:
    # df[feat] = (df[feat] / df["HuishoudensTotaal_28"])

In [ ]:
features = [
 'normalized_opgesteld_vermogen',
 'Gemeentenaam_1',
 'AantalInwoners_5',
 'WestersTotaal_17',
 'NietWestersTotaal_18',
 'HuishoudensTotaal_28',
 'GemiddeldeHuishoudensgrootte_32',
 'Bevolkingsdichtheid_33',
 'Woningvoorraad_34',
 #'GemiddeldeWoningwaarde_35',
 'PercentageEengezinswoning_36',
 'PercentageMeergezinswoning_37',
 'PercentageBewoond_38',
 'PercentageOnbewoond_39',
 'Koopwoningen_40',
 'InBezitWoningcorporatie_42',
 'InBezitOverigeVerhuurders_43',
 'BouwjaarVoor2000_45',
 'BouwjaarVanaf2000_46',
 'GemiddeldElektriciteitsverbruikTotaal_47',
 'GemiddeldAardgasverbruikTotaal_55',
 'PercentageWoningenMetStadsverwarming_63',
 'OpleidingsniveauLaag_64',
 'OpleidingsniveauMiddelbaar_65',
 'OpleidingsniveauHoog_66',
 'Nettoarbeidsparticipatie_67',
 'PercentageWerknemers_68',
 'PercentageZelfstandigen_69',
 'AantalInkomensontvangers_70',
 'GemiddeldInkomenPerInkomensontvanger_71',
 'GemiddeldInkomenPerInwoner_72',
 'GemGestandaardiseerdInkomenVanHuish_75',
 'BedrijfsvestigingenTotaal_91',
 'PersonenautoSTotaal_99',
 'PersonenautoSPerHuishouden_102',
 'PersonenautoSNaarOppervlakte_103',
 'Motorfietsen_104',
 'AfstandTotHuisartsenpraktijk_105',
 'AfstandTotGroteSupermarkt_106',
 'AfstandTotSchool_108',
 'MeestVoorkomendePostcode_113',
 'Dekkingspercentage_114',
 'MateVanStedelijkheid_115',
 'Omgevingsadressendichtheid_116']

In [ ]:
df = df[features]

In [ ]:
df.info()

In [ ]:
corr_matrix = df.corr().sort_values(by = 'normalized_opgesteld_vermogen', ascending = False).transpose()
corr_matrix = corr_matrix.sort_values(by = 'normalized_opgesteld_vermogen', ascending = False)

In [ ]:
plt.figure(figsize = (14,11))
sns.heatmap(data = corr_matrix, annot = False, fmt='.2f', cmap = 'RdBu_r', linewidths=.1, square=True, vmax=1, center = 0)

In [ ]:
plt.figure(figsize = (2,15))
sns.heatmap(data = corr_matrix[['normalized_opgesteld_vermogen']], 
            annot = True, fmt='.2f', cmap = 'RdBu_r', linewidths=.1, square=False, vmax=1, center = 0)

In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df_num = df.select_dtypes(include=numerics).drop('HuishoudensTotaal_28', axis = 1)

In [ ]:
fig, ax = plt.subplots(10, 4, figsize=(15,37), sharey=True, gridspec_kw={'hspace': 0.3})
for i, col in enumerate(df_num.columns):
    _ax=ax[i // 4, i % 4]
    sns.scatterplot(x=col, y='normalized_opgesteld_vermogen', data=df_num, ax=_ax)

In [ ]:
#df = df_num.copy()

In [ ]:
df.isna().sum()

In [ ]:
#df = df.dropna()

Kom pak je lasso maar

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso

In [ ]:
pipeline = Pipeline([
    ('scaler',StandardScaler()), 
    ('model',Lasso())])

In [ ]:
search = GridSearchCV(pipeline,
                      {'model__alpha':np.arange(0.01,100,0.01)},
                      cv = 5, 
                      scoring="neg_mean_squared_error",
                      verbose=3, error_score="raise")

See what values cannot be used in the lassoing.

In [ ]:
opgesteld_vermogen = 'normalized_opgesteld_vermogen'
feature_columns = df.loc[:, df.columns != opgesteld_vermogen]
print(len(df.columns))
print(len(feature_columns.columns))

X_train, X_test, y_train, y_test = train_test_split(feature_columns, df[opgesteld_vermogen], test_size=0.33, random_state=42)


In [ ]:
print("xtrain:", X_train.shape, "ytrain", y_train.shape, "xtest:", X_test.shape, "ytest:", y_test.shape)

In [ ]:
search.fit(X_train,y_train)

In [ ]:
search.best_params_

In [ ]:
coefficients = search.best_estimator_.named_steps['model'].coef_

In [ ]:
importance = np.abs(coefficients)
importance

In [ ]:
np.array(feature_columns.columns)[importance > 0]

In [ ]:
feature_columns.head()

## Create a first model - linear model (3 features)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

import altair as alt

In [ ]:
opgesteld_vermogen = 'normalized_opgesteld_vermogen'
columns_to_keep = [opgesteld_vermogen, 'GemiddeldInkomenPerInwoner_72', 'GemiddeldeHuishoudensgrootte_32', 'Koopwoningen_40']
df1 = df[columns_to_keep]

train_set, test_set = train_test_split(df1, test_size=0.2)

print(f"training set size: {len(train_set)}\ntest set size: {len(test_set)}")

Y_train_set = train_set[opgesteld_vermogen]
X_train_set = train_set.drop(opgesteld_vermogen, axis=1).copy()

Y_test_set = test_set[opgesteld_vermogen]
X_test_set = test_set.drop(opgesteld_vermogen, axis=1).copy()

lin_reg = LinearRegression()
lin_reg.fit(X_train_set, Y_train_set)

Plot the fit with income

In [ ]:
alt.renderers.enable('default')

base = alt.Chart(train_set).mark_circle().encode(
    alt.X('GemiddeldInkomenPerInwoner_72',
     title='# income'),
       alt.Y(opgesteld_vermogen,
     title='Opgesteld vermogen')
)

linear_fit = [
    base.transform_regression(
        "GemiddeldInkomenPerInwoner_72", opgesteld_vermogen, method="linear"
    )
    .mark_line()   
]

graph = alt.layer(base, *linear_fit)
graph

#### Check the quality of the model

In [ ]:
# Make predictions using the testing set
y_pred = lin_reg.predict(X_test_set)

# The coefficients
print("Coefficients: \n", lin_reg.coef_)
# The mean squared error
print("Mean squared error: %.3f" % mean_squared_error(Y_test_set, y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(Y_test_set, y_pred))

## Linear regression model with standard scaler (all the features)

In [ ]:
from sklearn.pipeline import make_pipeline, Pipeline 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LassoCV, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV         
from sklearn import metrics, set_config
set_config(display="diagram")
RANDOM_STATE = 42
TEST_SIZE = 0.5

In [ ]:
X = df.drop(['normalized_opgesteld_vermogen'], axis = 1)

In [ ]:
y = df['normalized_opgesteld_vermogen']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

In [ ]:
X_train.shape

In [ ]:
categorical_features = ['Gemeentenaam_1', 'MeestVoorkomendePostcode_113']

In [ ]:
numeric_features = X.columns.tolist()

In [ ]:
numeric_features.remove('Gemeentenaam_1')

In [ ]:
numeric_features.remove('MeestVoorkomendePostcode_113')

In [ ]:
numeric_features

In [ ]:
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

In [ ]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [ ]:
model_LR = Pipeline(steps=[('preprocessor', preprocessor),
                      ('estimator', LinearRegression())])

In [ ]:
model_LR.fit(X_train, y_train)

In [ ]:
y_pred = model_LR.predict(X_test)

In [ ]:
r2_score(y_test, y_pred)

## Random forest model

In [ ]:
model_RF = Pipeline(steps=[('preprocessor', preprocessor),
                      ('estimator', RandomForestRegressor(n_jobs=-1,
                                                          random_state=RANDOM_STATE,
                                                          min_weight_fraction_leaf=0.005))])

In [ ]:
model_RF.fit(X_train, y_train)

In [ ]:
y_pred = model_RF.predict(X_test)

In [ ]:
r2_score(y_test, y_pred)

## Create a first model - tree model

In [ ]:
from sklearn.tree import DecisionTreeRegressor

Create helper class

In [ ]:
class Result:
  def __init__(self, r_squared, mean_squared_error):
    self.r_squared = r_squared
    self.mean_squared_error = mean_squared_error

In [ ]:
results = {}

max_range = 10

# Fit regression models
for i in range(1, max_range):
    r = DecisionTreeRegressor(max_depth=i, random_state=3)
    r.fit(X_train_set, Y_train_set)
    y_predict = r.predict(X_test_set)
    results[i] = Result(r2_score(Y_test_set, y_predict), mean_squared_error(Y_test_set, y_predict)) 

#### Check the quality of the models

# Mark down

In [ ]:
for i in range(1,max_range):
    print("Depth  %2.f" % i, ":R squared: %.2f" % results[i].r_squared,
    "-- Mean squared error: %.0f" % results[i].mean_squared_error)


Best performance is at depth = 2 (Not the same over runs &#9785; though)

In [ ]:
optimum_depth = 5
print("Depth  %2.f" % optimum_depth, ":R squared: %.2f" % results[optimum_depth].r_squared,
    "-- Mean squared error: %.0f" % results[optimum_depth].mean_squared_error)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d0604020-40e6-4d7d-a2ba-74ef2b385723' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>